   # Shinnar-Le Roux RF Pulse Design with PulPy
  This notebook walks through the basics of PulPy SLR design, which is based on John Pauly's MATLAB SLR pulse design toolkit. Advanced methods such as pulse root-flipping and recursive SLR design are demonstrated as well.

In [ ]:
import numpy as np
import pulpy as pp
import pulpy.rf as rf


import sigpy.plot as pl
import scipy.signal as signal
import matplotlib.pyplot as pyplot

   ## Parameters for a time-bandwidth 4, linear-phase excitation

In [ ]:
tb = 8
N = 128
d1 = 0.01
d2 = 0.01
ptype = 'ex'
ftype = 'ls'

   ## Design the excitation pulse

In [ ]:
pulse = rf.slr.dzrf(N, tb, ptype, ftype, d1, d2, False)
pl.LinePlot(pulse, mode='r')

## Simulate the excitation pulse's Mxy profile

In [ ]:
[a, b] = pp.sim.abrm(pulse, np.arange(-2*tb, 2*tb, 0.01), True)
Mxy = 2*np.multiply(np.conj(a), b)
pl.LinePlot(Mxy)

## Parameters for a time-bandwidth 8, minimum-phase inversion

In [ ]:
tb = 8
N = 128
d1 = 0.01
d2 = 0.01
ptype = 'inv'
ftype = 'min'

## Design the inversion pulse

In [ ]:
pulse = rf.slr.dzrf(N, tb, ptype, ftype, d1, d2)
pl.LinePlot(pulse, mode='r')

## Simulate the inversion pulse's Mz profile

In [ ]:
[a, b] = pp.sim.abrm(pulse, np.arange(-2*tb, 2*tb, 0.01))
Mz = 1-2*np.abs(b)**2
pl.LinePlot(Mz.T, mode='r')
print(np.shape(Mz))

# Design a root-flipped saturation pulse

In [ ]:
tb = 12
N = 128
d1 = 0.01
d2 = 0.001
ptype = 'sat'
ftype = 'min'
# conventional pulse
pulse = rf.slr.dzrf(N, tb, ptype, ftype, d1, d2)
pl.LinePlot(pulse, mode='r')

In [ ]:
# root-flipped pulse
tb = 12
N = 128
d1 = 0.01
d2 = 0.001
flip = np.pi/2
ptype = 'sat'
[bsf, d1, d2] = rf.slr.calc_ripples(ptype, d1, d2)
b = bsf*rf.slr.dzmp(N, tb, d1, d2)
b = b[::-1]
[pulse, bRootFlipped] = rf.slr.root_flip(b, d1, flip, tb)

pyplot.figure()
pyplot.plot(np.abs(pulse))

# Recursive Pulse Design for a 3-Segment FLEET EPI scan

In [ ]:
# Design the pulses
Nseg = 3 # number of EPI segments/RF Pulses
tb = 4
N = 2000
seSeq = True
tbRef = 8 # time-bandwidth of ref pulse
[pulses, _] = rf.slr.dz_recursive_rf(Nseg, tb, N, seSeq, tbRef)

In [ ]:
# Plot them
pl.LinePlot(pulses.T, mode = 'real')

In [ ]:
# Simulate them
Mxy = np.zeros((np.size(np.arange(-4*tb, 4*tb, 0.01)), Nseg), dtype = complex)
Mz = np.ones(np.size(np.arange(-4*tb, 4*tb, 0.01)))
for ii in range(0, Nseg):    
    [a, b] = pp.sim.abrm(pulses[:, ii], np.arange(-4*tb, 4*tb, 0.01), True)
    Mxy[:, ii] = 2*Mz*np.multiply(np.conj(a),b)
    Mz = Mz*(1 - 2*np.abs(b)**2)

In [ ]:
# Plot Mxy profiles
pl.LinePlot(Mxy.T)